In [ ]:
# Exam 2020/06/27 - Exercise #2 - Version #2

In [ ]:
# Input data and output folders
inputPathPurchases = "exam_ex2_data/Purchases.txt"
outputPathPart1 = "outPart1v2/"
outputPathPart2 = "outPart2v2/"

# Write your code here

In [ ]:
# *****************************************
# Exercise 2 - Part 1
# *****************************************

In [ ]:
# Read the content of Purchases.txt
purchasesRDD = sc.textFile(inputPathPurchases).cache()

In [ ]:
# Select only the purchases related to years 2019
# MID,CustomerId,Date,Price
def year2019(line):
    fields = line.split(",")
    
    date = fields[2]
   
    if (date.startswith("2019")):
        return True
    else:
        return False
    
selectedPurchasesRDD = purchasesRDD.filter(year2019)    

In [ ]:
# Compute the income for each MID  

In [ ]:
# Extract pairs (BID,price)
# Map each input line to a pair
# key = MID
# value = price

def pairMIDPrice(line):
    # MID,CustomerId,Date,Price
    fields = line.split(",")
    mid = fields[0]
    price = float(fields[3])
    
    return (mid, price)
    
midsPricesRDD = selectedPurchasesRDD.map(pairMIDPrice)

In [ ]:
# Apply reduceByKey to compute the (annual) income for each MID  
midsIncomesRDD = midsPricesRDD.reduceByKey(lambda p1, p2: p1+p2).cache()

In [ ]:
# Select the maximum income in year 2019

In [ ]:
# Select only the annual incomes
incomesRDD = midsIncomesRDD.values()

In [ ]:
# Compute the maximum annual income
maxAnnualIncome = incomesRDD.reduce(lambda income1, income2: max(income1, income2))

In [ ]:
# Select the MIDs associated with the highest annual income in 2019
selectedMidsRDD = midsIncomesRDD.filter(lambda pair: pair[1]==maxAnnualIncome)

In [ ]:
# Store the result
selectedMidsRDD.keys().saveAsTextFile(outputPathPart1)

In [ ]:
# *****************************************
# Exercise 2 - Part 2
# *****************************************

In [ ]:
# Select the purchases of the last ten years (i.e., from year 2010 to year 2019) 
# MID,CustomerId,Date,Price
def year20102019(line):
    fields = line.split(",")
    
    date = fields[2]
    year = int(date.split("/")[0])
   
    if (year>=2010 and year<=2019):
        return True
    else:
        return False
    
selectedPurchases1019RDD = purchasesRDD.filter(year20102019)    

In [ ]:
# Compute the number of purchases for each MID in each year

In [ ]:
# Extract pairs (MID+year,+1)
# Map each input line to a pair
# key = (MID,year)
# value = +1

def pairMidYearOne(line):
    # MID,CustomerId,Date,Price
    fields = line.split(",")
    mid = fields[0]
    date = fields[2]
    year = int(date.split("/")[0])
   
    return ( (mid, year), 1)
    
midYearOneRDD = selectedPurchases1019RDD.map(pairMidYearOne)

In [ ]:
# Apply reduceByKey to compute the annual number of purchases for each MID in each year  
midYearNumPurchasesRDD = midYearOneRDD.reduceByKey(lambda p1, p2: p1+p2).cache()

In [ ]:
# Compute the maximum number of purchases (associated with one single model) per year

In [ ]:
# Extract pairs (year,annualPurchasesSingleModel)
# Map each input pair a pair
# key = year
# value = annualPurchasesSingleModel  
yearNumPurchasesRDD = midYearNumPurchasesRDD.map(lambda pair: (pair[0][1], pair[1]))

In [ ]:
# Apply reduceByKey to compute the maximum number of purchases (associated with one single model) per year
yearMaxNumPurchasesRDD = yearNumPurchasesRDD\
.reduceByKey(lambda numPurchases1, numPurchases2: max (numPurchases1, numPurchases2))

In [ ]:
# We are managing only 10 years. 
# Hence, the content of yearMaxNumPurchasesRDD can be stored in a local variable of the Driver
yearMaxNumPurchases = yearMaxNumPurchasesRDD.collectAsMap()

In [ ]:
# Select the pairs with 
# number of annual purchases per MID == maximum number of purchases (per single MID) in that year
def maxNumPurchases(pair):
    year = pair[0][1]
    numPurchases = pair[1]
    
    if numPurchases==yearMaxNumPurchases[year]:
        return True
    else:
        return False
    
midsMaxAnnualPurchasesRDD = midYearNumPurchasesRDD.filter(maxNumPurchases)

In [ ]:
# Count for each MID in how many years it is the most purchased model 

In [ ]:
# Map each input pair to a new pair
# key = MID
# value = +1
midOneRDD = midsMaxAnnualPurchasesRDD.map(lambda pair: (pair[0][0], 1))

In [ ]:
midNumYearsMostPurchasedRDD = midOneRDD.reduceByKey(lambda v1,v2: v1+v2)

In [ ]:
# Select the MIDs that have been the most purchased smartphone model in at least two years
selectedMIDsRDD = midNumYearsMostPurchasedRDD.filter(lambda pair: pair[1]>=2)

In [ ]:
# Store the selected MIDs
selectedMIDsRDD.keys().saveAsTextFile(outputPathPart2)